# Setting up the environment
It is assumed you are familiar with basics of the library. If not - see "Tutorial on basic topics".

In [1]:
import pymice as pm

pm.getTutorialData()


This is a bleeding edge version of the PyMICE library. It might meet your
expectations, however it might also go to your fridge, drink all the beer
it can find there and then eat your cat. Be warned.

https://www.dropbox.com/s/kxgfu7fazbwvz7e/C57_AB.zip?dl=1 data already downloaded.
All data already downloaded.



# Visit selection
## Visits of a particular mouse
To select visits of a particular mouse use the "mice" argument of getVisits method.

In [2]:
ml = pm.Loader('C57_AB/2012-08-28 15.33.58.zip')
visits = ml.getVisits(mice='C57 A 1')
print len(visits)
print visits[321].Animal

loading data from C57_AB/2012-08-28 15.33.58.zip
493
C57 A 1


## Visits of a group of mice
Multiple names can also be given as the "mice" argument.

In [3]:
visits = ml.getVisits(mice=['C57 A 1', 'C57 A 2'])
visitorNames = [v.Animal.Name for v in visits]
for mouse in set(visitorNames):
    print '%s: %d visits' % (mouse, visitorNames.count(mouse))

C57 A 1: 493 visits
C57 A 2: 281 visits


## Visits performed at particular time
To select visits before or after particular time - use "end" and "start" arguments, respectively.

In the example visits started between 15:00 (inclusive) and 16:00 (exclusive) 28th August 2012 are being selected.
**The time is given as _Eastern European Time_ (EET).**

In [4]:
from datetime import datetime
import pytz

EET = pytz.timezone('Etc/GMT-2')

timeFrom = datetime(2012, 8, 28, 15, 0, tzinfo=EET)
timeTo = datetime(2012, 8, 28, 16, 0, tzinfo=EET)

visits = ml.getVisits(start=timeFrom, end=timeTo)
print min(v.Start for v in visits)
print max(v.End for v in visits)


2012-08-28 15:34:14.140000+02:00
2012-08-28 16:00:00.187000+02:00


# Customising loaded data

## Omitting nosepoke data loading
In order to speed up the loading process you might disable loading of nosepoke data (if they are not necessary) by setting getNp option to False.

In [5]:
ml = pm.Loader('C57_AB/2012-08-28 15.33.58.zip', getNp=False)
visits = ml.getVisits(order=('Start', 'End'))
visit = visits[4]
print visit.Nosepokes

loading data from C57_AB/2012-08-28 15.33.58.zip
None


## Loading additional data
No log, environment nor hardware events data are loaded unless the loader is explicitely told to do so.

In [6]:
print ml.getLog()
print ml.getEnvironment()
print ml.getHardwareEvents()
ml = pm.Loader('C57_AB/2012-08-28 15.33.58.zip', getLog=True, getHw=True, getEnv=True)
print len(ml.getLog())
print len(ml.getEnvironment())
print len(ml.getHardwareEvents())

[]
[]
[]
loading data from C57_AB/2012-08-28 15.33.58.zip
1220
12273
177


# Data validity tests

It is important to be sure the data you are going to analyse are valid. For that purpose log analysis is necessary.

## Log analyzers
Log analysis is being performed on data being loaded (or merged) by a collection of log analyzers given as _`logAnalyzers`_ parameter of _`Loader`_ (or _`Merger`_  respectively) class constructor.

Instances of _`LickometerLogAnalyzer`_ class checks log for lickometer problems while _`PresenceLogAnalyzer`_ - for presence of unidentified animal.

If _`logAnalysers`_ parameter is not empty then it implies _`getLog`_ flag is _`True`_.

In [7]:
import glob
files = glob.glob('C57_AB/*.zip')

loaders = [pm.Loader(filename, getLog=True) for filename in files]
logAnalyzers = [pm.LickometerLogAnalyzer(),
                pm.PresenceLogAnalyzer(),
                ]
mm = pm.Merger(*loaders, logAnalyzers=logAnalyzers)

loading data from C57_AB/2012-08-28 13.44.51.zip
loading data from C57_AB/2012-08-28 15.33.58.zip
loading data from C57_AB/2012-08-31 11.46.31.zip
loading data from C57_AB/2012-08-31 11.58.22.zip


## Results of log analysis
After the analysis is performed, you can use _`TestMiceData`_ objects to check, if there is no problem with data in a period of interest (POI).

The POI in the example is defined as a period from first to last registration (obtained with _`getStart()`_ and _`getEnd()`_ methods).

In [8]:
start = mm.getStart()
end = mm.getEnd()
interval = (start, end)

strInterval = '(from %s to %s)' % interval

lickometerTester = pm.TestMiceData('Lickometer')
presenceTester = pm.TestMiceData('Presence')
if lickometerTester(mm, interval):
    print "lickometer ok " + strInterval
    
else:
    print "lickometer problem " + strInterval
    
if presenceTester(mm, interval):
    print "presences ok " + strInterval
    
else:
    print "unknown mice presence problem " + strInterval
    

lickometer problem (from 2012-08-28 13:44:52.671000+02:00 to 2012-09-13 13:35:34.390000+02:00)
presences ok (from 2012-08-28 13:44:52.671000+02:00 to 2012-09-13 13:35:34.390000+02:00)
